This note will be used for the data science capstone project

Create dataframe of Neighborhoods in Toronto

I'm using the same note book for week 3.

In [8]:
#imports
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd

In [9]:
#Getting the data from url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]               
df = pd.read_html(str(table))                       # read wiki table and put into a dataframe
data = pd.read_json(df[0].to_json(orient='records')) # update to allow for editing

In [10]:
data.head()

,Borough,Neighborhood,Postal Code
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A


In [11]:
#Choosing only data where field Borough doesn't have not assigned value
raw_data_selected = data[data['Borough'] != 'Not assigned']
#Grouping Data
raw_data_selected = raw_data_selected.groupby(['Borough', 'Postal Code'], as_index=False).agg(','.join)
raw_data_selected.head()

,Borough,Postal Code,Neighborhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


In [12]:
#Replacing values in Neighbourhood field with Borough where Neighbourhood is not assigned
raw_data_selected['Neighborhood'] = np.where(raw_data_selected['Neighborhood'] == 'Not assigned', raw_data_selected['Borough'], raw_data_selected['Neighborhood'])

# Final View of the Dataframe from question 1

In [16]:
dffinal = raw_data_selected[['Postal Code','Borough','Neighborhood']] # reset column order to align with requirement
dffinal.head()

,Postal Code,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M4P,Central Toronto,Davisville North
2,M4R,Central Toronto,"North Toronto West, Lawrence Park"
3,M4S,Central Toronto,Davisville
4,M4T,Central Toronto,"Moore Park, Summerhill East"


In [17]:
# Shape of DF
dffinal.shape

(103, 3)

# The below cells add the geospatial data to a new dataframe

In [18]:
# importing libraries

# using CSV file with geospatial data
!wget http://cocl.us/Geospatial_data
print('Data downloaded')

--2020-06-18 23:50:16--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.21, 159.8.69.24, 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.69.21|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-06-18 23:50:16--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.69.21|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-18 23:50:18--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-18 23:50:18--  https://ibm.box.com/public/static/

In [43]:
# converting CSV into dataframe
df_location = pd.read_csv('Geospatial_data')
df_location.columns = ['Postcode', 'Latitude', 'Longitude']

df_location.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## The below cells merge the neighborhoods in Toronto with their geospatial locations

In [46]:
# Merge new location data with original Toronto locations

merged_df = pd.merge(dffinal, df_location, 
                     left_on = 'Postal Code', 
                     right_on = 'Postcode', 
                     how='left')


# Remove duplicate Postcode from location DF 
merged_df.drop(['Postcode'], axis = 1, inplace = True) 
merged_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
